In [1]:
import json
import pandas as pd
import numpy as np
import re
import nltk

In [21]:
def processUser(tweet,df):
    userId = int(tweet['user']['id'])
    name = tweet['user']['screen_name']
    location = tweet['user']['location']
    url = tweet['user']['url']
    followers = tweet['user']['followers_count']
    lang = tweet['user']['lang']
    profile_image = tweet['user']['profile_image_url']
    following = tweet['user']['following']
    followers = tweet['user']['followers_count']
    df.loc[userId] = [name,location,url,followers,lang,profile_image,following,followers]

#Proceso de Tweets
def processTweet(tweet):
    tweet = tweet.lower()
    # www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #@username to AT_USER
    tweet = re.sub('@[^\s]+','AT_USER',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    return tweet


In [7]:
dfusers = pd.DataFrame(columns=['name','location','url','followers','lang','profile_image','following','followers'])
dftweets = pd.DataFrame(columns=['userId','geo','coordinates','timestamp','tweet','source','isretweeted','lang','lat','lon','polarity'])

In [8]:
file = open('C:\\Users\\Alberto\\PycharmProjects\\twitterExtraction\\CarmenaTweets.txt','r')
decoded = ''
counter = 0
for line in file:
    counter+=1
    
    try:
        if counter==1:
                print line
        decoded = json.loads(line)
    except Exception as e:
        print e,':',decoded['id'] 
        
    try:
        id = decoded['id']
        userId = str(decoded['user']['id'])
        #print decoded['user']['id']
        processUser(decoded, dfusers)
        geo = decoded['geo']
        coordinates = decoded['coordinates']
        timestamp = decoded['timestamp_ms']
        tweet = decoded['text']
        tweet = tweet.lower()
        # www.* or https?://* to URL
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
        #@username to AT_USER
        tweet = re.sub('@[^\s]+','AT_USER',tweet)
        #Remove additional white spaces
        tweet = re.sub('[\s]+', ' ', tweet)
        #Replace #word with word
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
        #trim
        tweet = tweet.strip('\'"') 
        source = decoded['source']
        isretweeted = False 
        if 'retweeted_status' in decoded:
            if decoded['retweeted_status'] is not 'null':
                isretweeted = True
        lang = decoded['lang']
        lat = 'null'
        lon = 'null'
        polarity = 'null'
        try:
            dftweets.loc[id] = [userId,geo,coordinates,timestamp,tweet,source,isretweeted,lang,lat,lon, polarity]
        except Exception as e:
            print e
    except Exception as a:
        print a.args,':','error processing tweet'

{"created_at":"Sun Nov 20 15:11:52 +0000 2016","id":800355989086203905,"id_str":"800355989086203905","text":"RT @elmundoes: El edificio (no okupado) que posee en Barcelona la concejal de Carmena que defiende a los okupas en Madrid\u2026 ","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":799032032017993733,"id_str":"799032032017993733","name":"GRUPO METRO MENDOZA","screen_name":"GRUPO_METRO_MZA","location":"Mendoza, Argentina","url":"http:\/\/www.grupometromza.com","description":"GRUPO METRO MENDOZA.\nPRODUCTORA DE SE\u00d1AL Y CONTENIDOS PARA RADIO Y TV . AGENCIA DE PUBLICIDAD Y PROMOCIONES PUBLICITARIAS.\nRADIOS AM Y FM . DIARIOS DIGITALES.","protected":false,"verified":false,"followers_count":24,"friends_count":353,"listed_count":5,"favourites_count"

In [9]:
dftweets.head(5)

,userId,geo,coordinates,timestamp,tweet,source,isretweeted,lang,lat,lon,polarity
800355989086203905,799032032017993733,None,None,1479654712350,rt AT_USER el edificio (no okupado) que posee ...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",True,es,null,null,null
800355991057530880,2326149346,None,None,1479654712820,rt AT_USER lo de alsasua no es terrorismo. esp...,"<a href=""http://twitter.com/download/iphone"" r...",True,es,null,null,null
800355991892037632,752958026,None,None,1479654713019,rt AT_USER el 'cara al sol' cierra la misa en ...,"<a href=""http://twitter.com/download/android"" ...",True,es,null,null,null
800355992147881988,2382475370,None,None,1479654713080,... podemos entrenar ahora si... quieres. no t...,"<a href=""http://twitter.com/download/android"" ...",False,es,null,null,null
800355997126512640,2405391592,None,None,1479654714267,rt AT_USER 😂😂😂😂😂👇 URL,"<a href=""http://twitter.com/download/android"" ...",True,und,null,null,null


In [10]:
len(dftweets)

7398

In [11]:
dfusers.head(5)

,name,location,url,followers,lang,profile_image,following
799032032017993733,GRUPO METRO MENDOZA,"Mendoza, Argentina",http://www.grupometromza.com,24.0,es,http://pbs.twimg.com/profile_images/7990356908...,None
2326149346,Pepe López,None,None,329.0,es,http://pbs.twimg.com/profile_images/4310926238...,None
752958026,XD,"Barcelona, España",None,62.0,es,http://pbs.twimg.com/profile_images/7344675448...,None
2382475370,Rαι Blαĸe,Estudiando,None,112.0,es,http://pbs.twimg.com/profile_images/7888527558...,None
2405391592,angel jurado sánchez,"Pedro Muñoz, Castilla-La Manch",None,304.0,es,http://pbs.twimg.com/profile_images/7508086238...,None


In [12]:
len(dfusers)

5573

In [13]:
len(dftweets[dftweets['lang']=='es'])

7317

In [14]:
dftweets.to_csv('tweets_df.csv', encoding='utf-8')

In [15]:
dfusers.to_csv('users_df.csv', encoding='utf-8')

## Carga a partir de DataFrame ya tratado

In [10]:
dftweets = pd.read_csv('tweets_df.csv', encoding='utf-8')

In [11]:
dfusers= pd.read_csv('users_df.csv', encoding='utf-8')

Cargamos el corpus tagueado

In [12]:
from lxml import objectify
xml = objectify.parse(open('general-tweets-train-tagged.xml'))
#sample tweet object
general_tweets_corpus_train = pd.DataFrame(columns=('content', 'polarity', 'agreement'))
root = xml.getroot()
tweets = root.getchildren()
for i in range(0,len(tweets)):
    tweet = tweets[i]
    row = dict(zip(['content', 'polarity', 'agreement'], [tweet.content.text, tweet.sentiments.polarity.value.text, tweet.sentiments.polarity.type.text]))
    row_s = pd.Series(row)
    row_s.name = i
    general_tweets_corpus_train = general_tweets_corpus_train.append(row_s)
general_tweets_corpus_train.to_csv('general-tweets-train-tagged.csv', index=False, encoding='utf-8')

In [13]:
general_tweets_corpus_train.head(5)

,content,polarity,agreement
0,"Salgo de #VeoTV , que día más largoooooo...",NONE,AGREEMENT
1,@PauladeLasHeras No te libraras de ayudar me/n...,NEU,DISAGREEMENT
2,@marodriguezb Gracias MAR,P,AGREEMENT
3,"Off pensando en el regalito Sinde, la que se v...",N+,AGREEMENT
4,Conozco a alguien q es adicto al drama! Ja ja ...,P+,AGREEMENT


In [14]:
tweets_corpus = general_tweets_corpus_train.copy()
tweets_corpus = tweets_corpus.query('agreement != "DISAGREEMENT" and polarity != "NONE"')
tweets_corpus.head(5)

,content,polarity,agreement
2,@marodriguezb Gracias MAR,P,AGREEMENT
3,"Off pensando en el regalito Sinde, la que se v...",N+,AGREEMENT
4,Conozco a alguien q es adicto al drama! Ja ja ...,P+,AGREEMENT
6,Toca @crackoviadeTV3 . Grabación dl especial N...,P+,AGREEMENT
8,Buen día todos! Lo primero mandar un abrazo gr...,P+,AGREEMENT


In [15]:
#remove links
tweets_corpus = tweets_corpus[-tweets_corpus.content.str.contains('^http.*$')]

tweets_corpus.shape

(4995, 3)

### Tokenizamos y hacemos Steming

### Evaluacion del modelo

In [16]:
general_tweets_corpus_train.ix[general_tweets_corpus_train['polarity']=='P+', 'polarity'] = 'P'
general_tweets_corpus_train.ix[general_tweets_corpus_train['polarity']=='N+', 'polarity'] = 'N'
general_tweets_corpus_train.ix[general_tweets_corpus_train['polarity']=='NONE', 'polarity'] = 'NEU'
general_tweets_corpus_train = general_tweets_corpus_train[general_tweets_corpus_train.polarity.str.contains("NEU") == False]
len(general_tweets_corpus_train)
#general_tweets_corpus_train[general_tweets_corpus_train['polarity']=='N'][0:2]

5066

In [17]:
subcorpus = []
general_tweets_corpus_train['words'] = general_tweets_corpus_train['content'].str.lower().str.split(' ')
general_tweets_corpus_train.head(5)
subcorpus = general_tweets_corpus_train[['words','polarity']]
tweets = [tuple(x) for x in subcorpus[0:1500].values]

In [18]:
def get_words_in_tweets(tweets):
    all_words = []
    for (words, sentiment) in tweets:
        all_words.extend(words)
    return all_words


def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

word_features = get_word_features(get_words_in_tweets(tweets))

def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [19]:
from nltk.tokenize import word_tokenize
from nltk.classify import NaiveBayesClassifier

# get the training set and train the Naive Bayes Classifier
training_set = nltk.classify.util.apply_features(extract_features, tweets)

In [20]:
NBClassifier = nltk.NaiveBayesClassifier.train(training_set)

In [50]:
import nltk


#print NBClassifier.classify(extract_features(getFeatureVector(processedTweet('RT @Misssione: Madrid lleno de hojas ca\u00eddas que con la lluvia, te resbalas. \u00bfSe har\u00e1 cargo Carmena y su equipo de los da\u00f1os que suframos lo\u2026')))) + '\n'
dftest = dftweets[0:20]

dftestcomplete = dftest
     
for i in range(len(dftweets)):
    #print dftest.ix[i]['tweet']
    dftweets.set_value(i, 'polarity', NBClassifier.classify(extract_features(nltk.word_tokenize(processTweet(dftweets.ix[i]['tweet'])))))
    #print dftest.ix[i]['polarity'] 


In [51]:
dftweets.to_csv('tweets_analyzed.csv', encoding='utf-8')

In [85]:
import matplotlib.pyplot as plt

len(dftweets[dftweets['polarity']=='P'])
len(dftweets[dftweets['polarity']=='N'])

len(dftweets[dftweets['lat']=='null'])
len(dftweets[dftweets['lon']=='null'])

dftweets['coordinates'].unique()
dftweets['geo'].unique()



0

In [87]:
dftweets[['tweet','polarity']].head(50)

,tweet,polarity
0,rt AT_USER el edificio (no okupado) que posee ...,P
1,rt AT_USER lo de alsasua no es terrorismo. esp...,N
2,rt AT_USER el 'cara al sol' cierra la misa en ...,N
3,... podemos entrenar ahora si... quieres. no t...,P
4,rt AT_USER 😂😂😂😂😂👇 URL,P
5,rt AT_USER el edificio (no okupado) que posee ...,P
6,rt AT_USER el edificio (no okupado) que posee ...,P
7,rt AT_USER el edificio (no okupado) que posee ...,P
8,rt AT_USER el edificio (no okupado) que posee ...,P
9,rt AT_USER el 'cara al sol' cierra la misa en ...,N
